In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis


In [192]:
from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES, get_bt_index

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.fgw import encode_target, FGW

from rdkit.Chem import MolFromSmiles

import torch
from torch import nn
import numpy as np
import time

In [3]:
data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 48, True)
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [4]:
sys.argv = ["", "-cuda"]
args = get_args()
args.output_dir = "mol_opt/output/"

mol_opt = MolOpt(args)
mol_opt_decoder = MolOptDecoder(args)
loss = FGW(alpha = 0.5)

In [198]:
x_embedding, x_delta_hat = mol_opt.forward(X)
yhat_embedding = x_embedding + x_delta_hat
yhat_logits = mol_opt_decoder.forward(yhat_embedding, Y)
yhat_labels = mol_opt_decoder.discretize(*yhat_logits)
loss((yhat_labels, yhat_logits, Y.scope), Y)

tensor(180.8488, device='cuda:0', grad_fn=<AddBackward0>)

In [6]:
symbols_logits, charges_logits, bonds_logits = yhat_logits
symbols_nll, charges_nll, bonds_nll = -nn.LogSoftmax(dim=1)(symbols_logits), -nn.LogSoftmax(dim=1)(charges_logits), -nn.LogSoftmax(dim=1)(bonds_logits)

In [25]:
symbols_nll.shape

torch.Size([1061, 64])

In [31]:
yhat_labels[0]

tensor([25,  7,  7,  ...,  7,  7,  7], device='cuda:0',
       grad_fn=<NotImplemented>)

In [22]:
symbols_nll

tensor([[4.1151, 4.1547, 4.2164,  ..., 4.0651, 4.0776, 4.2221],
        [4.1128, 4.1475, 4.2505,  ..., 4.0518, 4.0424, 4.2092],
        [4.1492, 4.1749, 4.2679,  ..., 4.0830, 4.0894, 4.2525],
        ...,
        [4.1439, 4.1996, 4.2309,  ..., 4.0577, 4.0508, 4.2307],
        [4.1914, 4.1981, 4.1970,  ..., 4.0729, 4.0927, 4.2033],
        [4.1494, 4.1610, 4.1817,  ..., 4.0803, 4.0675, 4.1877]],
       device='cuda:0', grad_fn=<NegBackward>)

In [8]:
charges_nll.shape

torch.Size([1061, 5])

In [35]:
bonds_nll.shape

torch.Size([23733, 5])

In [100]:
# get encoding of symbols
symbols_lst = [ys.symbol for y in Y.mols for ys in y.atoms]
target_symbols = torch.zeros_like(symbols_nll)
for idxl, l in enumerate(symbols_lst):
    for idxs, s in enumerate(SYMBOLS):
        if (s == l):
            symbols_lst[idxl] = idxs
            target_symbols[idxl, idxs] = 1

In [85]:
bonds_lst = 

torch.Size([1061])

In [115]:
target_bonds = torch.zeros_like(bonds_nll)
target_bonds[:,-1].fill_(1)

In [170]:
target_bonds

tensor([[0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        ...,
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], device='cuda:0')

In [203]:
[fatoms, fbonds, gromov_agraph, bgraph], scope = Y.gromov_get_graph_inputs()

In [174]:
len(Y.mols[3].atoms)

26

In [169]:
# encode bonds
bond_idx = 0
for mol_idx, mol in enumerate(Y.mols):
    _, lex = Y.scope[mol_idx]
    for x in mol.bonds:
        curr_bond_idx = bond_idx + lex * x.in_atom_idx + x.out_atom_idx
        target_bonds[curr_bond_idx, -1] = 0
        target_bonds[curr_bond_idx, get_bt_index(x.bond_type)] = 1

    bond_idx += lex ** 2

In [167]:
target_bonds[-20:]

tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], device='cuda:0')

In [166]:
bond_idx

23733

In [162]:
target_bonds.shape

torch.Size([23733, 5])

In [161]:
sum([x[1] ** 2 for x in Y.scope])

23733

In [151]:
[(x.in_atom_idx, x.out_atom_idx, x.bond_type) for x in mol.bonds]

[(1, 0, rdkit.Chem.rdchem.BondType.SINGLE),
 (0, 1, rdkit.Chem.rdchem.BondType.SINGLE),
 (2, 1, rdkit.Chem.rdchem.BondType.SINGLE),
 (1, 2, rdkit.Chem.rdchem.BondType.SINGLE),
 (3, 2, rdkit.Chem.rdchem.BondType.SINGLE),
 (2, 3, rdkit.Chem.rdchem.BondType.SINGLE),
 (4, 3, rdkit.Chem.rdchem.BondType.SINGLE),
 (3, 4, rdkit.Chem.rdchem.BondType.SINGLE),
 (5, 4, rdkit.Chem.rdchem.BondType.SINGLE),
 (4, 5, rdkit.Chem.rdchem.BondType.SINGLE),
 (6, 5, rdkit.Chem.rdchem.BondType.SINGLE),
 (5, 6, rdkit.Chem.rdchem.BondType.SINGLE),
 (7, 6, rdkit.Chem.rdchem.BondType.SINGLE),
 (6, 7, rdkit.Chem.rdchem.BondType.SINGLE),
 (8, 7, rdkit.Chem.rdchem.BondType.SINGLE),
 (7, 8, rdkit.Chem.rdchem.BondType.SINGLE),
 (9, 8, rdkit.Chem.rdchem.BondType.DOUBLE),
 (8, 9, rdkit.Chem.rdchem.BondType.DOUBLE),
 (10, 8, rdkit.Chem.rdchem.BondType.SINGLE),
 (8, 10, rdkit.Chem.rdchem.BondType.SINGLE),
 (11, 10, rdkit.Chem.rdchem.BondType.SINGLE),
 (10, 11, rdkit.Chem.rdchem.BondType.SINGLE),
 (12, 11, rdkit.Chem.rdche

In [63]:
y.bonds[0].out_atom_idx

0

In [168]:
get_bt_index(x.bond_type)

3

In [109]:
len(BOND_TYPES) - 1

(array([0, 1, 2, 3]),)

In [180]:
encode_target(Y)

(tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         ...,
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.]]))

In [183]:
loss = FGW(alpha = 0.5)

In [191]:
loss((yhat_labels, yhat_logits, Y.scope), Y)

tensor(180.8488, device='cuda:0', grad_fn=<AddBackward0>)

In [188]:
Y.scope

[(0, 18),
 (18, 24),
 (42, 19),
 (61, 26),
 (87, 19),
 (106, 24),
 (130, 20),
 (150, 17),
 (167, 20),
 (187, 25),
 (212, 25),
 (237, 25),
 (262, 24),
 (286, 22),
 (308, 23),
 (331, 22),
 (353, 22),
 (375, 19),
 (394, 24),
 (418, 19),
 (437, 20),
 (457, 20),
 (477, 23),
 (500, 26),
 (526, 26),
 (552, 23),
 (575, 23),
 (598, 24),
 (622, 21),
 (643, 20),
 (663, 25),
 (688, 23),
 (711, 24),
 (735, 22),
 (757, 21),
 (778, 21),
 (799, 17),
 (816, 26),
 (842, 20),
 (862, 22),
 (884, 21),
 (905, 24),
 (929, 21),
 (950, 19),
 (969, 24),
 (993, 24),
 (1017, 22),
 (1039, 22)]